## Set Up the Environment

In [1]:
import os

In [1]:
%pip install transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tf-keras

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 13.3 MB/s eta 0:00:00
Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl (390.2 MB)
Using cached keras-3.6.0-py3-none-any.whl (1.2 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled ker


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install sentencepiece sacremoses

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------------------------- 991.5/991.5 kB 15.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   --------------------------------------- 897.5/897.5 kB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install transformers[torch]

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.5.1-cp311-cp311-win_amd64.whl (203.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load the Dataset

In [5]:
from datasets import load_dataset

ds = load_dataset("wmt/wmt14", "de-en")
print(ds)

c:\Users\AIML\anaconda3\envs\aiml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})


## Loading the Tokenizer

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset

# Load the tokenizer and model
model_name = "Helsinki-NLP/opus-mt-de-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

## Preprocess the Data

In [7]:
# Preprocess data
def preprocess_function(examples):
    inputs = [ex["de"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = ds.map(preprocess_function, batched=True)

## Model and Training Arguments

In [8]:
batch_size = 16

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./opus-mt-de-en-finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

c:\Users\AIML\anaconda3\envs\aiml\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Initialize the Trainer and Fine-Tune

In [9]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

C:\Users\AIML\AppData\Local\Temp\ipykernel_37468\2171734158.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
  0%|          | 500/281800 [00:39<7:03:12, 11.08it/s]c:\Users\AIML\anaconda3\envs\aiml\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 1.3394, 'grad_norm': 1.3751311302185059, 'learning_rate': 1.9964939673527327e-05, 'epoch': 0.0}


  0%|          | 1000/281800 [01:25<6:57:36, 11.21it/s]

{'loss': 0.883, 'grad_norm': 1.5851682424545288, 'learning_rate': 1.992945351312988e-05, 'epoch': 0.0}


  1%|          | 1500/281800 [02:11<6:57:35, 11.19it/s] 

...

100%|█████████▉| 281500/281800 [6:47:50<00:28, 10.61it/s]

{'loss': 0.4752, 'grad_norm': 1.1099728345870972, 'learning_rate': 3.0305180979418026e-08, 'epoch': 1.0}


                                                         
100%|██████████| 281800/281800 [6:48:29<00:00, 11.50it/s]

{'eval_loss': 0.32258763909339905, 'eval_runtime': 7.401, 'eval_samples_per_second': 405.351, 'eval_steps_per_second': 25.402, 'epoch': 1.0}
{'train_runtime': 24509.8665, 'train_samples_per_second': 183.958, 'train_steps_per_second': 11.497, 'train_loss': 0.5145373834592387, 'epoch': 1.0}


TrainOutput(global_step=281800, training_loss=0.5145373834592387, metrics={'train_runtime': 24509.8665, 'train_samples_per_second': 183.958, 'train_steps_per_second': 11.497, 'total_flos': 1.5284043931189248e+17, 'train_loss': 0.5145373834592387, 'epoch': 1.0})

## Save the Fine-Tuned Model

In [10]:
model.save_pretrained("./opus-mt-de-en-finetuned")
tokenizer.save_pretrained("./opus-mt-de-en-finetuned")

('./opus-mt-de-en-finetuned\\tokenizer_config.json',
 './opus-mt-de-en-finetuned\\special_tokens_map.json',
 './opus-mt-de-en-finetuned\\vocab.json',
 './opus-mt-de-en-finetuned\\source.spm',
 './opus-mt-de-en-finetuned\\target.spm',
 './opus-mt-de-en-finetuned\\added_tokens.json')